In [7]:
import torch, detectron2
import matplotlib.pyplot as plt
from detectron2.utils.logger import setup_logger
#from defisheye import Defisheye

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import cv2
import numpy as np
import os
from pathlib import Path

setup_logger()
torch.cuda.empty_cache()

import numpy as np
import os
import json

def mask_to_annotation_pixels(mask, step=3):
    """
    Converts a boolean mask to x, y coordinates of polygon points by selecting every nth pixel.
    
    :param mask: 2D boolean numpy array
    :param step: Step to pick points from the mask (default: 3)
    :return: all_points_x, all_points_y
    """
    all_points_x = []
    all_points_y = []
    
    # Get non-zero pixel indices (y, x) from the mask
    y_indices, x_indices = np.where(mask)
    area = np.sum(mask)
    print(area)
    if area < 1000:
        return None, None
    # Skip masks with no valid points
    if len(x_indices) == 0 or len(y_indices) == 0:
        return None, None

    # Select every nth point based on the step
    all_points_x = x_indices[::step].tolist()
    all_points_y = y_indices[::step].tolist()

    return all_points_x, all_points_y

def get_file_size(file_path):
    """
    Returns the file size of an image.
    
    :param file_path: Path to the image filef
    :return: File size in bytes
    """
    return os.path.getsize(file_path)

def load_existing_json(output_file):
    """
    Loads existing JSON data with proper encoding and error handling.
    
    :param output_file: Path to the JSON file
    :return: List of JSON annotations
    """
    if os.path.exists(output_file):
        try:
            with open(output_file, "r", encoding="utf-8") as f:
                return json.load(f)
        except UnicodeDecodeError as e:
            print(f"Error reading JSON file: {e}")
            return []
    return []

def convert_annotations_to_json(fine_name, predictions):
    annotations = {}
    annotations["filename"] = fine_name
    annotations["size"] = 0  # You can set this to the size of the image file if needed
    annotations["regions"] = []
    global total_instances
    for i in range(len(predictions["instances"].pred_classes)):
        region = {}
        mask = predictions["instances"].pred_masks[i].cpu().numpy()
        area = np.sum(mask)
        #print("Area : ",area)
        if area < 3000:
            continue
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if len(contour) > 2:  # Only consider valid polygons
                region["shape_attributes"] = {
                    "name": "polygon",
                    "all_points_x": contour[:, 0, 0].tolist(),
                    "all_points_y": contour[:, 0, 1].tolist()
                }
                
                region["region_attributes"] = {
                    # "label": MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes[predictions["instances"].pred_classes[i]]
                    "label": "cow"
                }
                
                annotations["regions"].append(region)
        
                total_instances += 1
                #print("total instances : ", total_instances)

    return annotations

def write_annotation_json(annotations, file_path):

    # file_ = f"{file_name}_auto_annotation.json"
    # file_path = os.path.join(output_dir, file_)
    
    # if not os.path.exists(output_dir):
    #     os.makedirs(output_dir)

    with open(file_path, "w") as f:
        json.dump(annotations, f, indent=4)

def append_annotation_json(output_file, filename, file_size,mask, step=3):
    """
    Appends the annotation for the given mask to a JSON file in the specified format.
    
    :param output_file: Path to the JSON file
    :param filename: Image filename
    :param mask: Boolean mask (2D numpy array)
    :param step: Step for pixel selection
    """
    global total_instances
    
    polygon_x, polygon_y = mask_to_annotation_pixels(mask, step)

    # Skip if there are no valid y-values
    if polygon_x is None or polygon_y is None:
        print(f"Skipping mask for {filename} because no valid polygon points were found.")
        return

    # Get file size

    total_instances += 1
    # Create a dictionary for the current annotation
    annotation = {
        "filename": filename,
        "file_size": file_size,
        "file_attributes": {},
        "region_count": 1,
        "region_id": 0,
        "region_shape_attributes": {
            "name": "polygon",
            "all_points_x": polygon_x,
            "all_points_y": polygon_y
        },
        "region_attributes": {
            "cow": "cow"
        }
    }

    # Load existing data or create an empty list if the file doesn't exist
    data = load_existing_json(output_file)
    print("Writing total instanace No: ",total_instances)
    # Append the new annotation to the list
    data.append(annotation)

    # Save the updated data back to the JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def resize_detect_and_annotate(folder,output_file,step=3):
    images = get_files_from_folder(folder)
    all_annotations = {}
    for image in images:
        try:
            
        
            image_path = os.path.join(folder, image)
            #print(image_path)
            #resize_image_dir = folder + "/resize/" +image
            frame = cv2.imread(image_path)
            #if 'Cam6' in image_path:
            #frame = cv2.resize(frame,(1152,768))
            #else:
            #    frame = cv2.resize(frame,(640,480))
            #cv2.imwrite(resize_image_dir,frame)
            #file_size = get_file_size(resize_image_dir)
            
            #colored_mask = np.zeros_like(frame)
            outputs = predictor(frame)
            all_annotations[image]=convert_annotations_to_json(image, outputs)
        except Exception as ex:
            print(ex)
    write_annotation_json(all_annotations,output_file)

def detect_and_annotate(folder, output_file, step=3):
    """
    Processes a single image and its corresponding mask, then appends the annotation to a JSON file.
    
    :param image_path: Path to the image file
    :param mask: Boolean mask (2D numpy array)
    :param output_file: Path to the output JSON file
    :param step: Step for pixel selection
    """
    # Assuming the mask is already generated and passed to this function
    images = get_files_from_folder(folder)
    all_annotations = {}
    for image in images:
        image_path = Path(os.path.join(folder, image))
        file_size = get_file_size(image_dir)
        frame = cv2.imread(image_path)
        colored_mask = np.zeros_like(frame)
        outputs = predictor(frame)
        all_annotations[image]=convert_annotations_to_json(image, outputs)
        #break
    write_annotation_json(all_annotations,output_file)


def configure_model(model_path):
    cfg = get_cfg()
    cfg.set_new_allowed(True) #added this to solve the below problem // allow merging other config
    # # loading initial weight file
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.merge_from_file(f"{model_path}\\config.yml")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75 #0.6
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4 #0.3
    # cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.WEIGHTS = f"{model_path}\\model_best.pth"
    # If you don't have a GPU and CUDA enabled, the next line is required
    # cfg.MODEL.DEVICE = "cpu"
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    
    predictor = DefaultPredictor(cfg)
    return predictor

#model_path = "models\\Feb 2025\\07_Feb_2025_15000_iters_v1"
#model_path = "models\\April 2025\\April_Day_Night_10000_iters_v2" //latest April 2025 
#model_path = "models\\April 2025\\April_Day_Night_10000_base_Sumiyoshi_dec31_v1_human_clear_2024_dec_10000_iters_v2_iters_v2" #better mask April 2025 best better mask
#model_path = "models\\April 2025\\April_Day_Night_15000_base_Sumiyoshi_2024_10000_iters_v4_iters_v1" 
model_path = "models\\Side Lane August 2025\Base_rtx8000_10_August_2025_20000_v1" 
#model_path = "models\\base_Sumiyoshi_model_Sumiyoshi_anchor_v1_10000_iters1" #october 2 report
#model_path = "models\\base_Detection_Model_T6_model_Sumiyoshi_anchor_v2_10000_iters1"
predictor = configure_model(model_path)
#detect_and_annotate(image_dir,output_file)

[09/24 21:08:29 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from models\Side Lane August 2025\Base_rtx8000_10_August_2025_20000_v1\model_best.pth ...


In [10]:
#model_path = "models\\October 2024\\Sumiyoshi_oct2_2024_10000_iters_v4" #october 2 report
#model_path = "models\\base_yoshii_model_Sumiyoshi_anchor_v1_15000_iters1"
# Example usage:
cam_path = '1_Annotate_24_Sept_KNP'
image_dir = rf"D:\Nyi Zaw Aung\815_CowDataChecking\Sumiyoshi ToAnnotate\August 2025\Daytime_2025-09-24\\{cam_path}"
output_file = fr"{image_dir}\{cam_path}_json.json"
total_instances = 0
#model_path = "models\\October 2024\\Sumiyoshi_oct2_2024_10000_iters_v2" #october 2 report  Sumiyoshi_oct2_2024_2000_iters_v1




In [11]:
resize_detect_and_annotate(image_dir,output_file)

In [ ]:
images_path = [r"C:\Users\choni\Pictures\5-51.png",
                r"C:\Users\choni\Pictures\5-52.png",
                r"C:\Users\choni\Pictures\5-53.png",
                r"C:\Users\choni\Pictures\5-54.png"]
    
frames = []
for image_path in images_path:
    frame = cv2.imread(image_path)
    frames.append(frame)
results = predictor(frames)
for result in results:
    print("total boxes : ",len(result["instances"].pred_boxes))

In [ ]:

model = 'models\\March 2025\\Feb_March_Day_Night_15000_iters_v2/model_final.pth'
model_config = 'models\\March 2025\\Feb_March_Day_Night_15000_iters_v2/config.yml'


filelist = []
filelist = [r"C:\Users\choni\Pictures\5-51.png",
            r"C:\Users\choni\Pictures\5-52.png",
            r"C:\Users\choni\Pictures\5-53.png",
            r"C:\Users\choni\Pictures\5-54.png"]
# for root, dirs, files in os.walk(root):
#     for file in files:
#         filelist.append(Path(os.path.join(root, file)))

#with open(classes) as classes_file:
class_names = {1}

cfg = get_cfg()
cfg.merge_from_file(cfg_filename=model_config)
cfg.MODEL.WEIGHTS = model
print("Initializing model")
pred = BatchPredictor(cfg=cfg, classes=class_names, batch_size=4, workers=0)
print("Finsished initializing model")   
t1 = time.time()
predictions = list(pred(filelist[0:3]))
t2 = time.time()
print(f'Time used: {t2-t1} seconds')